In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45010
2,Huelva,Confirmados PDIA 14 días,430
3,Huelva,Tasa PDIA 14 días,"83,79289514196076"
4,Huelva,Confirmados PDIA 7 días,174
5,Huelva,Tasa PDIA 7 dias,"33,90689245279342"
6,Huelva,Total Confirmados,45211
7,Huelva,Curados,42422
8,Huelva,Fallecidos,411


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45010.0


/tmp/ipykernel_3344/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12458.0


/tmp/ipykernel_3344/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3344/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3344/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3344/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3345 personas en los últimos 7 días 

Un positivo PCR cada 1635 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45010.0,174.0,430.0,513170.0,33.906892,83.792895,23.0
Huelva-Costa,26646.0,122.0,305.0,289548.0,42.134637,105.336594,13.0
Condado-Campiña,13960.0,37.0,82.0,156231.0,23.682880,52.486382,8.0
Huelva (capital),12458.0,43.0,88.0,143837.0,29.894951,61.180364,6.0
Ayamonte,2039.0,21.0,45.0,21104.0,99.507202,213.229719,3.0
Palos de la Frontera,1010.0,9.0,15.0,11742.0,76.647931,127.746551,3.0
Cartaya,1942.0,25.0,72.0,20083.0,124.483394,358.512174,2.0
Almonte,2262.0,9.0,22.0,24507.0,36.724201,89.770270,1.0
Bonares,456.0,2.0,4.0,6060.0,33.003300,66.006601,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Nava (La),9.0,2.0,2.0,258.0,775.193798,775.193798,0.0
Cañaveral de León,33.0,1.0,2.0,396.0,252.525253,505.050505,0.0
Cartaya,1942.0,25.0,72.0,20083.0,124.483394,358.512174,2.0
Aroche,128.0,1.0,9.0,3054.0,32.743942,294.695481,0.0
Sanlúcar de Guadiana,35.0,1.0,1.0,403.0,248.138958,248.138958,0.0
Higuera de la Sierra,65.0,1.0,3.0,1291.0,77.459334,232.378002,0.0
Cumbres Mayores,135.0,3.0,4.0,1749.0,171.526587,228.702115,1.0
Isla Cristina,3102.0,16.0,46.0,21393.0,74.790819,215.023606,1.0
Ayamonte,2039.0,21.0,45.0,21104.0,99.507202,213.229719,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Aroche,128.0,1.0,9.0,3054.0,32.743942,294.695481,0.0,0.111111
Punta Umbría,1255.0,1.0,6.0,15355.0,6.512537,39.075220,0.0,0.166667
San Juan del Puerto,758.0,1.0,6.0,9411.0,10.625863,63.755180,0.0,0.166667
Valverde del Camino,833.0,1.0,4.0,12750.0,7.843137,31.372549,0.0,0.250000
Aljaraque,1612.0,1.0,3.0,21474.0,4.656794,13.970383,0.0,0.333333
Higuera de la Sierra,65.0,1.0,3.0,1291.0,77.459334,232.378002,0.0,0.333333
Cartaya,1942.0,25.0,72.0,20083.0,124.483394,358.512174,2.0,0.347222
Isla Cristina,3102.0,16.0,46.0,21393.0,74.790819,215.023606,1.0,0.347826
Lepe,2985.0,11.0,28.0,27880.0,39.454806,100.430416,1.0,0.392857
